include

In [2]:
# -*- encoding: utf-8 -*-
# siemese.py
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.utils import data
import cv2
import json
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy
from PIL import Image
import pandas as pd
from ast import Add
from logging import critical
from multiprocessing import reduction
from turtle import forward

#from siamese_net import SiameseNetwork
#from CustomDataset import CustomDataset


In [2]:
import shutil
shutil.copy("../input/myproject-model/model (6).pth","./model.pth")

'./model.pth'

In [15]:
from IPython.display import FileLink
FileLink('model.pth')

/kaggle/working/model.pth

In [1]:
#callbacks.py
class LossHistory():
    def __init__(self, log_dir, model, input_shape):
        time_str        = datetime.datetime.strftime(datetime.datetime.now(),'%Y_%m_%d_%H_%M_%S')
        self.log_dir    = os.path.join(log_dir, "loss_" + str(time_str))
        self.losses     = []
        self.val_loss   = []
        
        os.makedirs(self.log_dir)
        self.writer     = SummaryWriter(self.log_dir)
        try:
            dummy_input     = torch.randn(2, 2, 3, input_shape[0], input_shape[1])
            self.writer.add_graph(model, dummy_input)
        except:
            pass

    def append_loss(self, epoch, loss, val_loss):
        if not os.path.exists(self.log_dir):
            os.makedirs(self.log_dir)

        self.losses.append(loss)
        self.val_loss.append(val_loss)

        with open(os.path.join(self.log_dir, "epoch_loss.txt"), 'a') as f:
            f.write(str(loss))
            f.write("\n")
        with open(os.path.join(self.log_dir, "epoch_val_loss.txt"), 'a') as f:
            f.write(str(val_loss))
            f.write("\n")

        self.writer.add_scalar('loss', loss, epoch)
        self.writer.add_scalar('val_loss', val_loss, epoch)
        self.loss_plot()

    def loss_plot(self):
        iters = range(len(self.losses))

        plt.figure()
        plt.plot(iters, self.losses, 'red', linewidth = 2, label='train loss')
        plt.plot(iters, self.val_loss, 'coral', linewidth = 2, label='val loss')
        try:
            if len(self.losses) < 25:
                num = 5
            else:
                num = 15
            
            plt.plot(iters, scipy.signal.savgol_filter(self.losses, num, 3), 'green', linestyle = '--', linewidth = 2, label='smooth train loss')
            plt.plot(iters, scipy.signal.savgol_filter(self.val_loss, num, 3), '#8B4513', linestyle = '--', linewidth = 2, label='smooth val loss')
        except:
            pass

        plt.grid(True)
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend(loc="upper right")

        plt.savefig(os.path.join(self.log_dir, "epoch_loss.png"))

        plt.cla()
        plt.close("all")

训练网络

In [3]:
# all
average=torch.tensor(80.6786)
variance=torch.tensor(9.0062)
class SiameseNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn1 = nn.Sequential(
            nn.BatchNorm2d(1,affine=True), # B*C*H*W
            nn.Conv2d(1, 64, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2)
            # B * 64 * 32 * 32
        )
        self.cnn2 = nn.Sequential(
            nn.BatchNorm2d(64,affine=True), # B*C*H*W
            nn.Conv2d(64, 128, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            # nn.Conv2d(128, 128, kernel_size=5, padding=2),
            # nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2)
            # B * 128 * 16 * 16
        )
        self.cnn3 = nn.Sequential(
            nn.BatchNorm2d(128,affine=True), # B*C*H*W
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            # nn.ReLU(inplace=True),
            # nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2)
            # B * 256 * 8 * 8
        )
        self.cnn4 = nn.Sequential(
            nn.BatchNorm2d(256,affine=True), # B*C*H*W
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2),
            # B * 512 * 4 * 4
        )

        self.fc1 = nn.Sequential(
#             nn.Linear(512 * 4 * 4, 100, bias=True),
            nn.Linear(256*8*8,1024,bias=True),
#             nn.Linear(1000, 100, bias=True),
            nn.ReLU(inplace=True),
            nn.Linear(1024,1024,bias=True),
            # nn.Linear(20, 1)
        )

        self.fc2 = nn.Sequential(
#             nn.Linear(100, 20),
#             nn.ReLU(inplace=True),
#             nn.Linear(20,1),
            nn.Linear(1024,100),
            nn.ReLU(inplace=True),
            nn.Linear(100,1),
        )
        # for m in self.modules():
        #     if isinstance(m,nn.Conv2d):
        #         nn.init.kaiming_normal_(m.weight.data,0.3)
        #         if m.bias is not None:
        #             nn.init.constant_(m.bias.data,0.3)
        #     elif isinstance(m,nn.Linear):
        #         nn.init.kaiming_normal_(m.weight.data)
        #         if m.bias is not None:
        #             nn.init.constant_(m.bias.data,0.5)
        #     elif isinstance(m,nn.BatchNorm2d):
        #         nn.init.kaiming_normal_(m.weight.data,0.3)
        #         if m.bias is not None:
        #             nn.init.constant_(m.bias.data,0)
    def forward_once(self, x):
        B, C, H, W = x.shape
        output = self.cnn1(x)
        output = self.cnn2(output)
        output = self.cnn3(output)
#         output = self.cnn4(output)
#         output = output.view(B, 512*4*4)
        output=output.view(B,256*8*8)
        output = self.fc1(output)
#         output = output.view(B,100)
#         output = nn.BatchNorm1d(100,eps=1e-4,affine=True).cuda()(output) # B*C*H*W
#         output = nn.Sigmoid()(output)
        return output

    def forward(self, input1, input2):
        B, C, H, W = input1.shape
        output1 = self.forward_once(input1)
        # 7*20
        output2 = self.forward_once(input2)
#         output3 = nn.functional.cosine_similarity(output1, output2, dim=1)-torch.ones((1,B)).cuda()*0.995
#         output3=torch.abs(output2-output1).sum(dim=1)
#         output3=(output1-output2).norm(2,dim=1,keepdim=True)
        output3=torch.abs(output1-output2)
        output3=self.fc2(output3)
#         output3 = output3.resize(B, 1)
        output4=output3
        output4=output4*variance+average
#         output4=100*(torch.ones(B,1).cuda()-output4)
        output4=torch.min(output4,torch.ones(B,1).cuda()*100)
        return output4
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, path):
        super(CustomDataset,self).__init__()
        # path="E:/MyProject/"
        self.path0="../input/standard/standard/"
        self.path1="../input/train1/train1/"
        self.path2="../input/train2/train2/"
        self.index0=json.loads(open("../input/index-standard/index_standard.json",'r',encoding='utf-8').read())
        self.index1=json.loads(open("../input/index-train1/index_train1.json",'r',encoding='utf-8').read())
        self.index2=json.loads(open("../input/index-train2/index_train2.json",'r',encoding='utf-8').read())
        self.dframe0=pd.DataFrame(self.index0)
#         self.len1=len(self.index1)//4
        self.len1=0
        self.len2=len(self.index2)*6
    def __getitem__(self,index):
        if (index<self.len1):
            word=self.index1[index]["word"]
            filename1=self.path1+self.index1[index]["filename"]
            score=self.index1[index]["score"]
        else:
            index-=self.len1
            if (index<len(self.index2)*6):
                word=self.index2[index//6]["word"]
                filename1=self.path2+self.index1[index//6]["filename"]
                score=self.index2[index//6]["score"]
        item = self.dframe0.loc[self.dframe0['word']==word]
        fileName = item["filename"].values[0]
        filename2 = self.path0 + fileName
        #filename2=self.path0+self.dframe0.loc[self.dframe0["word"]==word]["filename"][1]
        
        input_one = Image.open(filename2)
        input_one_np = numpy.array(input_one)
        input_one=numpy.where(numpy.sum(input_one_np,axis=2)==3*255,0,1)
        # print(input_one.shape)

        input_two = Image.open(filename1)
        input_two_np = numpy.array(input_two)
        input_two=numpy.where(input_two_np==255,0,1)
        # print(input_two.shape)
        
        
        input_one_tensor = torch.tensor(input_one, dtype=torch.float32).unsqueeze(0)
        input_two_tensor = torch.tensor(input_two, dtype=torch.float32).unsqueeze(0)
        label_tensor = torch.tensor(int(score),dtype=torch.float32)
        # input_one.cuda()
        # input_two.cuda()
        # label.cuda()
        input_one_tensor=transforms.Compose([transforms.RandomAffine(degrees=5,translate=(0.05,0.05)),transforms.ColorJitter(brightness=[0.4,1.0],contrast=[0.7,1.1])])(input_one_tensor)
        input_two_tensor=transforms.Compose([transforms.RandomAffine(degrees=5,translate=(0.05,0.05)),transforms.ColorJitter(brightness=[0.4,1.0],contrast=[0.7,1.1])])(input_two_tensor)
        return input_one_tensor,input_two_tensor, label_tensor

    def __len__(self):
#         len_1=len(self.index1)//4
#         len_1=0
#         len_2=len(self.index2)*6
        return self.len1+self.len2

# train.py
dataset = CustomDataset(path="E:/MyProject/")
print(len(dataset))
#train_dataset,test_dataset=data.random_split(dataset,torch(len(dataset)/100*95),len(dataset)-torch.floor(len(dataset)/100*95))
n_train = int(len(dataset)/100*95)
n_val = len(dataset) - n_train
train_dataset,test_dataset=data.random_split(dataset,[n_train, n_val])
print(len(train_dataset))
print(len(test_dataset))
train_dataloader=data.DataLoader(
    train_dataset,
    batch_size=64,    
    shuffle=True,
    #num_worders=8,
    pin_memory=True,
    drop_last=True
)
test_dataloader=data.DataLoader(
    test_dataset,
    batch_size=1,
    #num_worders=8,
    shuffle=True,
    pin_memory=True
)


# net

siamese=torch.load("./model.pth")
# siamese=SiameseNetwork()
siamese=siamese.cuda()
#siamese =SiameseNetwork().cuda() #定义模型以致GPU

# loss
# loss_func = ContrastiveLoss() #定义损失函数
loss_func = torch.nn.MSELoss()
#----------------------#
#   获得损失函数
#----------------------#
#loss_func = torch.nn.BCEWithLogitsLoss()
#----------------------#
#   记录Loss
#----------------------#
# if local_rank == 0:
#     loss_history = LossHistory(save_dir, model, input_shape=input_shape)
# else:
#     loss_history = None

average=torch.tensor(83.0401)
variance=torch.tensor(10.2655)

train_list=[siamese.cnn1,siamese.cnn2,siamese.cnn3,siamese.fc1,siamese.fc2]
def train(a,b):
    params=[
        {"params":siamese.cnn1.parameters(),"lr":a},
        {"params":siamese.cnn2.parameters(),"lr":a},
        {"params":siamese.cnn3.parameters(),"lr":a},
        {"params":siamese.cnn4.parameters(),"lr":a},
        {"params":siamese.fc1.parameters(),"lr":b},
        {"params":siamese.fc2.parameters(),"lr":b},
    ]
    optimizer=torch.optim.Adam(params) # Adam对学习率不太敏感
    lost_sum=0
    for i, (image1,image2,labels) in enumerate(train_dataloader): # enumerate 将一个可遍历的数据对象组合为一个索引序列
        image1=image1.cuda()
        image2=image2.cuda()
        
        labels=labels.cuda()
#         print(image1.shape)
#         print(image2.shape)
        
        # print(image1.shape, image2.shape)
        optimizer.zero_grad()
        outputs=siamese(image1,image2)
        # print(outputs.shape, labels.shape)
        loss=loss_func(outputs,labels)
        lost_sum=lost_sum+((outputs-labels)*(outputs-labels)).sum()
        loss.backward()
        optimizer.step()
#         print("epoch is {}, ite is {}/{}, loss is {}".format(epoch+1, i, len(train_dataset) // 64, loss.item()))
    torch.save(siamese,"./model.pth")
    lost_sum=lost_sum/len(train_dataset)
    print(lost_sum.sqrt())
# training

for epoch in range(50): #epoch one usage of whole dataset
    train(1e-5,1e-5)
    train(1e-5,0)
    train(0,1e-5)
        
    loss_test=0
    lst=[]
    for i,(image1,image2,labels) in enumerate(test_dataloader):
        image1=image1.cuda()
        image2=image2.cuda()
        outputs=siamese(image1,image2)
        outputs=outputs.cpu()
        labels=labels.to(torch.float32)
        #[batchsize]
        #outputs=batchsize*cls_num
        now=(outputs-labels)*(outputs-labels)
        lst.append(outputs)
        loss_test += now
        if (i % 40 ==0):
            print(outputs,labels)
        # _, pred=outputs.max(dim=1)
    loss_test=torch.sqrt(loss_test/len(test_dataloader))
    print("epoch is {}, loss test is {}".format(epoch+1,loss_test.item()))
    
    lst=torch.tensor(lst)
    print("mean={},std={}".format(torch.mean(lst),torch.std(lst)))
    # input("next")

torch.save(siamese,"./model.pth")

# eval/test
# save

# load
# inference

7038
6686
352


/opt/conda/lib/python3.7/site-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


tensor(72.0779, device='cuda:0', grad_fn=<SqrtBackward0>)
tensor(72.0335, device='cuda:0', grad_fn=<SqrtBackward0>)
tensor(72.0603, device='cuda:0', grad_fn=<SqrtBackward0>)
tensor([[80.7151]], grad_fn=<ToCopyBackward0>) tensor([83.])
tensor([[80.5497]], grad_fn=<ToCopyBackward0>) tensor([90.])
tensor([[80.6833]], grad_fn=<ToCopyBackward0>) tensor([70.])
tensor([[80.5771]], grad_fn=<ToCopyBackward0>) tensor([80.])
tensor([[80.5092]], grad_fn=<ToCopyBackward0>) tensor([63.])
tensor([[80.7235]], grad_fn=<ToCopyBackward0>) tensor([89.])
tensor([[80.5851]], grad_fn=<ToCopyBackward0>) tensor([79.])
tensor([[80.4518]], grad_fn=<ToCopyBackward0>) tensor([70.])
tensor([[80.8379]], grad_fn=<ToCopyBackward0>) tensor([93.])
epoch is 1, loss test is 8.5984525680542
mean=80.70970916748047,std=0.1797911822795868
tensor(72.0507, device='cuda:0', grad_fn=<SqrtBackward0>)
tensor(72.0672, device='cuda:0', grad_fn=<SqrtBackward0>)
tensor(72.0204, device='cuda:0', grad_fn=<SqrtBackward0>)
tensor([[80.6340

KeyboardInterrupt: 

使用普通的VGG网络，效果一般

预训练res18

In [ ]:
# -*- encoding: utf-8 -*-
# resnet18
# siemese.py
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.utils import data
import cv2
import json
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy
from PIL import Image
import pandas as pd
from ast import Add
from logging import critical
from multiprocessing import reduction


#from siamese_net import SiameseNetwork
#from CustomDataset import CustomDataset
from urllib.request import urlopen
from torchvision.models import resnet18
average=torch.tensor(80.6786)
variance=torch.tensor(9.0062)
class SiameseNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        
        basenet=resnet18(pretrained=True)
        self.conv_5=nn.Sequential(
            nn.Sequential(
                basenet.conv1,
                basenet.bn1,
                basenet.relu,
                basenet.maxpool
            ),
            basenet.layer1,
            basenet.layer1,
            basenet.layer1,
            basenet.layer1,
        )
#         self.stage0=nn.Sequential(
#             basenet.conv1,
#             basenet.bn1,
#             basenet.relu,
#             basenet.maxpool
#         )
#         self.stage1=basenet.layer1
#         self.stage2=basenet.layer1
#         self.stage3=basenet.layer1
#         self.stage4=basenet.layer1
        
        self.fc1 = nn.Sequential(
            nn.Linear(16*16*64,256,bias=True),
            nn.ReLU(inplace=True),
            # nn.Linear(20, 1)
        )

        self.fc2 = nn.Sequential(
#             nn.Linear(100, 20),
#             nn.ReLU(inplace=True),
#             nn.Linear(20,1),
            nn.Linear(512,100),
            nn.ReLU(inplace=True),
            nn.Linear(100,1),
            nn.Sigmoid(),
        )
    def forward_once(self, x):
        B, C, H, W = x.shape
        x=torch.cat([x,x,x],dim=1)
        f=self.conv_5(x)
#         print(f.shape)
        output=f.view(B,16*16*64)
        output = self.fc1(output)
#         output = output.view(B,100)
#         output = nn.BatchNorm1d(100,eps=1e-4,affine=True).cuda()(output) # B*C*H*W
#         output = nn.Sigmoid()(output)
        return output

    def forward(self, input1, input2):
        B, C, H, W = input1.shape
        output1 = self.forward_once(input1)
        # 7*20
        output2 = self.forward_once(input2)
        output3=torch.cat([output1,output2],dim=1)
        output3=self.fc2(output3)
#         output3 = output3.resize(B, 1)
        output4=output3
        output4=output4*100
#         output4=output4*variance+average
#         output4=100*(torch.ones(B,1).cuda()-output4)
#         output4=torch.min(output4,torch.ones(B,1).cuda()*100)
        return output4
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, path):
        super(CustomDataset,self).__init__()
        # path="E:/MyProject/"
        self.path0="../input/standard/standard/"
        self.path1="../input/train1/train1/"
        self.path2="../input/train2/train2/"
        self.index0=json.loads(open("../input/index-standard/index_standard.json",'r',encoding='utf-8').read())
        self.index1=json.loads(open("../input/index-train1/index_train1.json",'r',encoding='utf-8').read())
        self.index2=json.loads(open("../input/index-train2/index_train2.json",'r',encoding='utf-8').read())
        self.dframe0=pd.DataFrame(self.index0)
#         self.len1=len(self.index1)//4
        self.len1=0
        self.len2=len(self.index2)*6
    def __getitem__(self,index):
        if (index<self.len1):
            word=self.index1[index]["word"]
            filename1=self.path1+self.index1[index]["filename"]
            score=self.index1[index]["score"]
        else:
            index-=self.len1
            if (index<len(self.index2)*6):
                word=self.index2[index//6]["word"]
                filename1=self.path2+self.index1[index//6]["filename"]
                score=self.index2[index//6]["score"]
        item = self.dframe0.loc[self.dframe0['word']==word]
        fileName = item["filename"].values[0]
        filename2 = self.path0 + fileName
        #filename2=self.path0+self.dframe0.loc[self.dframe0["word"]==word]["filename"][1]
        
        input_one = Image.open(filename2)
        input_one_np = numpy.array(input_one)
        input_one=numpy.where(numpy.sum(input_one_np,axis=2)==3*255,0,1)
        # print(input_one.shape)

        input_two = Image.open(filename1)
        input_two_np = numpy.array(input_two)
        input_two=numpy.where(input_two_np==255,0,1)
        # print(input_two.shape)
        
        
        input_one_tensor = torch.tensor(input_one, dtype=torch.float32).unsqueeze(0)
        input_two_tensor = torch.tensor(input_two, dtype=torch.float32).unsqueeze(0)
        label_tensor = torch.tensor(int(score),dtype=torch.float32)
        # input_one.cuda()
        # input_two.cuda()
        # label.cuda()
        input_one_tensor=transforms.Compose([transforms.RandomAffine(degrees=5,translate=(0.05,0.05)),transforms.ColorJitter(brightness=[0.4,1.0],contrast=[0.7,1.1])])(input_one_tensor)
        input_two_tensor=transforms.Compose([transforms.RandomAffine(degrees=5,translate=(0.05,0.05)),transforms.ColorJitter(brightness=[0.4,1.0],contrast=[0.7,1.1])])(input_two_tensor)
        return input_one_tensor,input_two_tensor, label_tensor

    def __len__(self):
#         len_1=len(self.index1)//4
#         len_1=0
#         len_2=len(self.index2)*6
        return self.len1+self.len2

# train.py
dataset = CustomDataset(path="E:/MyProject/")
print(len(dataset))
#train_dataset,test_dataset=data.random_split(dataset,torch(len(dataset)/100*95),len(dataset)-torch.floor(len(dataset)/100*95))
n_train = int(len(dataset)/100*95)
n_val = len(dataset) - n_train
train_dataset,test_dataset=data.random_split(dataset,[n_train, n_val])
print(len(train_dataset))
print(len(test_dataset))
train_dataloader=data.DataLoader(
    train_dataset,
    batch_size=64,    
    shuffle=True,
    #num_worders=8,
    pin_memory=True,
    drop_last=True
)
test_dataloader=data.DataLoader(
    test_dataset,
    batch_size=1,
    #num_worders=8,
    shuffle=True,
    pin_memory=True
)


# net

try:
    siamese=torch.load("./model_res18.pth")
except:
    siamese=SiameseNetwork()
siamese=siamese.cuda()
#siamese =SiameseNetwork().cuda() #定义模型以致GPU

# loss
loss_func = torch.nn.MSELoss()

average=torch.tensor(83.0401)
variance=torch.tensor(10.2655)

def train(a,b):
    params=[
        {"params":siamese.conv_5.parameters(),"lr":a},
        {"params":siamese.fc1.parameters(),"lr":b},
        {"params":siamese.fc2.parameters(),"lr":b},
    ]
    optimizer=torch.optim.Adam(params) # Adam对学习率不太敏感
    lost_sum=0
    for i, (image1,image2,labels) in enumerate(train_dataloader): # enumerate 将一个可遍历的数据对象组合为一个索引序列
        image1=image1.cuda()
        image2=image2.cuda()
        
        labels=labels.cuda()
#         print(image1.shape)
#         print(image2.shape)
        
        # print(image1.shape, image2.shape)
        optimizer.zero_grad()
        outputs=siamese(image1,image2)
        # print(outputs.shape, labels.shape)
        loss=loss_func(outputs,labels)
        lost_sum=lost_sum+((outputs-labels)*(outputs-labels)).sum()
        loss.backward()
        optimizer.step()
#         print("epoch is {}, ite is {}/{}, loss is {}".format(epoch+1, i, len(train_dataset) // 64, loss.item()))
    torch.save(siamese,"./model_res18.pth")
    lost_sum=lost_sum/len(train_dataset)
    print(lost_sum.sqrt())
# training

for epoch in range(10): #epoch one usage of whole dataset
#     train(1e-5,1e-5)
#     train(1e-5,0)
#     train(0,1e-5)
    train(1e-5,1e-3)
        
    loss_test=0
    lst=[]
    for i,(image1,image2,labels) in enumerate(test_dataloader):
        image1=image1.cuda()
        image2=image2.cuda()
        outputs=siamese(image1,image2)
        outputs=outputs.cpu()
        labels=labels.to(torch.float32)
        #[batchsize]
        #outputs=batchsize*cls_num
        now=(outputs-labels)*(outputs-labels)
        lst.append(outputs)
        loss_test += now
        if (i % 40 ==0):
            print(outputs,labels)
        # _, pred=outputs.max(dim=1)
    loss_test=torch.sqrt(loss_test/len(test_dataloader))
    print("epoch is {}, loss test is {}".format(epoch+1,loss_test.item()))
    
    lst=torch.tensor(lst)
    print("mean={},std={}".format(torch.mean(lst),torch.std(lst)))
    # input("next")

torch.save(siamese,"./model_res18.pth")

# eval/test
# save

# load
# inference

7038
6686
352


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

/opt/conda/lib/python3.7/site-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


tensor(170.6405, device='cuda:0', grad_fn=<SqrtBackward0>)
tensor([[100.]], grad_fn=<ToCopyBackward0>) tensor([78.])
tensor([[100.]], grad_fn=<ToCopyBackward0>) tensor([74.])
tensor([[100.]], grad_fn=<ToCopyBackward0>) tensor([86.])
tensor([[100.]], grad_fn=<ToCopyBackward0>) tensor([79.])
tensor([[100.]], grad_fn=<ToCopyBackward0>) tensor([86.])
tensor([[100.]], grad_fn=<ToCopyBackward0>) tensor([89.])
tensor([[100.]], grad_fn=<ToCopyBackward0>) tensor([76.])
tensor([[100.]], grad_fn=<ToCopyBackward0>) tensor([60.])
tensor([[100.]], grad_fn=<ToCopyBackward0>) tensor([88.])
epoch is 1, loss test is 22.097858428955078
mean=100.0,std=0.0
tensor(169.7705, device='cuda:0', grad_fn=<SqrtBackward0>)
tensor([[100.]], grad_fn=<ToCopyBackward0>) tensor([65.])
tensor([[100.]], grad_fn=<ToCopyBackward0>) tensor([60.])
tensor([[100.]], grad_fn=<ToCopyBackward0>) tensor([90.])
tensor([[100.]], grad_fn=<ToCopyBackward0>) tensor([87.])
tensor([[100.]], grad_fn=<ToCopyBackward0>) tensor([82.])
tensor(

In [ ]:
# 计算平均值，方差
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, path):
        super(CustomDataset,self).__init__()
        # path="E:/MyProject/"
        self.path0="../input/standard/standard/"
        self.path1="../input/train1/train1/"
        self.path2="../input/train2/train2/"
        self.index0=json.loads(open("../input/index-standard/index_standard.json",'r',encoding='utf-8').read())
        self.index1=json.loads(open("../input/index-train1/index_train1.json",'r',encoding='utf-8').read())
        self.index2=json.loads(open("../input/index-train2/index_train2.json",'r',encoding='utf-8').read())
        self.dframe0=pd.DataFrame(self.index0)
#         self.len1=len(self.index1)//4
        self.len1=0
        self.len2=len(self.index2)*6
    def __getitem__(self,index):
        if (index<self.len1):
            word=self.index1[index]["word"]
            filename1=self.path1+self.index1[index]["filename"]
            score=self.index1[index]["score"]
        else:
            index-=self.len1
            if (index<len(self.index2)*6):
                word=self.index2[index//6]["word"]
                filename1=self.path2+self.index1[index//6]["filename"]
                score=self.index2[index//6]["score"]
        item = self.dframe0.loc[self.dframe0['word']==word]
        fileName = item["filename"].values[0]
        filename2 = self.path0 + fileName
        #filename2=self.path0+self.dframe0.loc[self.dframe0["word"]==word]["filename"][1]
        
        input_one = Image.open(filename2)
        input_one_np = numpy.array(input_one)
        input_one=numpy.where(numpy.sum(input_one_np,axis=2)==3*255,0,1)
        # print(input_one.shape)

        input_two = Image.open(filename1)
        input_two_np = numpy.array(input_two)
        input_two=numpy.where(input_two_np==255,0,1)
        # print(input_two.shape)
        
        
        input_one_tensor = torch.tensor(input_one, dtype=torch.float32).unsqueeze(0)
        input_two_tensor = torch.tensor(input_two, dtype=torch.float32).unsqueeze(0)
        label_tensor = torch.tensor(int(score),dtype=torch.float32)
        # input_one.cuda()
        # input_two.cuda()
        # label.cuda()
        input_one_tensor=transforms.Compose([transforms.RandomAffine(degrees=5,translate=(0.05,0.05)),transforms.ColorJitter(brightness=[0.4,1.0],contrast=[0.7,1.1])])(input_one_tensor)
        input_two_tensor=transforms.Compose([transforms.RandomAffine(degrees=5,translate=(0.05,0.05)),transforms.ColorJitter(brightness=[0.4,1.0],contrast=[0.7,1.1])])(input_two_tensor)
        return input_one_tensor,input_two_tensor, label_tensor

    def __len__(self):
#         len_1=len(self.index1)//4
#         len_1=0
#         len_2=len(self.index2)*6
        return self.len1+self.len2

dataset = CustomDataset(path="E:/MyProject/")
l=len(dataset)
cal1=torch.tensor(0)
cal2=torch.tensor(0)
for i in range(l):
    a,b,c=dataset.__getitem__(i)
    cal1=cal1+c
average=cal1/l
for i in range(l):
    a,b,c=dataset.__getitem__(i)
    cal2=cal2+(c-average)*(c-average)
cal2=cal2/l
cal2=torch.sqrt(cal2)
print(average,cal2)

In [ ]:
average=torch.tensor(83.0401)
variance=torch.tensor(10.2655)
class SiameseNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn1 = nn.Sequential(
            nn.BatchNorm2d(1,affine=True), # B*C*H*W
            nn.Conv2d(1, 64, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2)
            # B * 64 * 32 * 32
        )
        self.cnn2 = nn.Sequential(
            nn.BatchNorm2d(64,affine=True), # B*C*H*W
            nn.Conv2d(64, 128, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            # nn.Conv2d(128, 128, kernel_size=5, padding=2),
            # nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2)
            # B * 128 * 16 * 16
        )
        self.cnn3 = nn.Sequential(
            nn.BatchNorm2d(128,affine=True), # B*C*H*W
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            # nn.ReLU(inplace=True),
            # nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2)
            # B * 256 * 8 * 8
        )
        self.cnn4 = nn.Sequential(
            nn.BatchNorm2d(256,affine=True), # B*C*H*W
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2)
            # B * 512 * 4 * 4
        )

        self.fc1 = nn.Sequential(
            nn.Linear(512 * 4 * 4, 100, bias=True),
#             nn.Linear(1000, 100, bias=True),
            # nn.ReLU(inplace=True),
            # nn.Linear(20, 1)
        )

        self.fc2 = nn.Sequential(
            nn.Linear(100, 1, bias=False),
        )
        # for m in self.modules():
        #     if isinstance(m,nn.Conv2d):
        #         nn.init.kaiming_normal_(m.weight.data,0.3)
        #         if m.bias is not None:
        #             nn.init.constant_(m.bias.data,0.3)
        #     elif isinstance(m,nn.Linear):
        #         nn.init.kaiming_normal_(m.weight.data)
        #         if m.bias is not None:
        #             nn.init.constant_(m.bias.data,0.5)
        #     elif isinstance(m,nn.BatchNorm2d):
        #         nn.init.kaiming_normal_(m.weight.data,0.3)
        #         if m.bias is not None:
        #             nn.init.constant_(m.bias.data,0)
    def forward_once(self, x):
        B, C, H, W = x.shape
        output = self.cnn1(x)
        output = self.cnn2(output)
        output = self.cnn3(output)
        output = self.cnn4(output)
        output = output.view(B, 512*4*4)
        output = self.fc1(output)
#         output = output.view(B,100)
#         output = nn.BatchNorm1d(100,eps=1e-4,affine=True).cuda()(output) # B*C*H*W
#         output = nn.Sigmoid()(output)
        return output

    def forward(self, input1, input2):
        B, C, H, W = input1.shape
        output1 = self.forward_once(input1)
        # 7*20
        output2 = self.forward_once(input2)
#         output3 = nn.functional.cosine_similarity(output1, output2, dim=1)-torch.ones((1,B)).cuda()*0.995
#         output3=torch.abs(output2-output1).sum(dim=1)
#         output3=(output1-output2).norm(2,dim=1,keepdim=True)
        output3=output1-output2
        output3=self.fc2(output3)
#         output3 = output3.resize(B, 1)
        output4=output3
        output4=output4*variance+average
        return output4
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, path):
        super(CustomDataset,self).__init__()
        # path="E:/MyProject/"
        self.path0="../input/standard/standard/"
        self.path1="../input/train1/train1/"
        self.path2="../input/train2/train2/"
        self.index0=json.loads(open("../input/index-standard/index_standard.json",'r',encoding='utf-8').read())
        self.index1=json.loads(open("../input/index-train1/index_train1.json",'r',encoding='utf-8').read())
        self.index2=json.loads(open("../input/index-train2/index_train2.json",'r',encoding='utf-8').read())
        self.dframe0=pd.DataFrame(self.index0)
#         self.len1=len(self.index1)//4
        self.len1=0
        self.len2=len(self.index2)*6
    def __getitem__(self,index):
        if (index<self.len1):
            word=self.index1[index]["word"]
            filename1=self.path1+self.index1[index]["filename"]
            score=self.index1[index]["score"]
        else:
            index-=self.len1
            if (index<len(self.index2)*6):
                word=self.index2[index//6]["word"]
                filename1=self.path2+self.index1[index//6]["filename"]
                score=self.index2[index//6]["score"]
        item = self.dframe0.loc[self.dframe0['word']==word]
        fileName = item["filename"].values[0]
        filename2 = self.path0 + fileName
        #filename2=self.path0+self.dframe0.loc[self.dframe0["word"]==word]["filename"][1]
        
        input_one = Image.open(filename2)
        input_one_np = numpy.array(input_one)
        input_one=numpy.where(numpy.sum(input_one_np,axis=2)==3*255,0,1)
        # print(input_one.shape)

        input_two = Image.open(filename1)
        input_two_np = numpy.array(input_two)
        input_two=numpy.where(input_two_np==255,0,1)
        # print(input_two.shape)
        
        
        input_one_tensor = torch.tensor(input_one, dtype=torch.float32).unsqueeze(0)
        input_two_tensor = torch.tensor(input_two, dtype=torch.float32).unsqueeze(0)
        label_tensor = torch.tensor(int(score),dtype=torch.float32)
        # input_one.cuda()
        # input_two.cuda()
        # label.cuda()
        input_one_tensor=transforms.Compose([transforms.RandomAffine(degrees=5,translate=(0.05,0.05)),transforms.ColorJitter(brightness=[0.4,1.0],contrast=[0.7,1.1])])(input_one_tensor)
        input_two_tensor=transforms.Compose([transforms.RandomAffine(degrees=5,translate=(0.05,0.05)),transforms.ColorJitter(brightness=[0.4,1.0],contrast=[0.7,1.1])])(input_two_tensor)
        return input_one_tensor,input_two_tensor, label_tensor

    def __len__(self):
#         len_1=len(self.index1)//4
#         len_1=0
#         len_2=len(self.index2)*6
        return self.len1+self.len2

dataset = CustomDataset(path="E:/MyProject/")
print(len(dataset))
dataloader=data.DataLoader(
    dataset,
    batch_size=1,
    #num_worders=8,
    shuffle=True,
    pin_memory=True
)

# net

siamese=torch.load("./model.pth").cpu()
print (len(dataset))
accuracy=0
for i,(image1,image2,labels) in enumerate(dataloader):
    
    outputs=siamese(image1,image2)
    pred=outputs

    labels=labels.cpu().numpy()[0]
    pred=pred.cpu().detach().numpy()[0]
    # batchsize*1*28*28
    # 通道数（灰度图=1） 在显示时应放在最后

    print("label",labels)
    print("pred",pred)

accuracy=torch.sqrt(accuracy/len(dataset))
# 方差
print("accuracy is {}".format(accuracy))
      

运行顺序：shutil-1 -> import -> train/all -> shutil-2
        shutil-1 -> import -> test -> shutil-2